**Application of combined physiological masks**

In [19]:
# ============================================================
# APPLY COMBINED MASKS (ON/OFF-BODY + VAN HEES) TO EDA, ACC, IBI
# FOR ALL SUBJECTS (EMBRACE + EMPATICA)
# ============================================================

import os
import numpy as np
import pandas as pd

# ============================================================
# SUBJECT PAIRING (EMBRACE ↔ EMPATICA)
# ============================================================

PAIRING = {
    "CLIDEM01": "1823370", "CLIDEM010": "1842113", "CLIDEM011": "1891004",
    "CLIDEM02": "1832743", "CLIDEM030": "1916330", "CLIDEM031": "1920720",
    "CLIDEM04": "1833368", "CLIDEM05": "1837341", "CLIDEM06": "1840660",
    "CLIDEM07": "1840605", "CLIDEM08": "1841430", "CLIDEM09": "1842162",
    "CLIDEM11": "1891004", "CLIDEM12": "1891236", "CLIDEM13": "1895868",
    "CLIDEM14": "1891785", "CLIDEM15": "1894131", "CLIDEM16": "1895849",
    "CLIDEM17": "1896649", "CLIDEM18": "1898066", "CLIDEM19": "1900189",
    "CLIDEM20": "1906938", "CLIDEM21": "1907236", "CLIDEM22": "1908525",
    "CLIDEM23": "1909384", "CLIDEM24": "1910353", "CLIDEM25": "1911867",
    "CLIDEM26": "1912600", "CLIDEM27": "1913365", "CLIDEM29": "1914654"
}

# ============================================================
# 1. LOAD AND COMBINE MASKS (ON/OFF-BODY + VAN HEES)
# ============================================================

def load_combined_mask(embrace_id, empatica_id,
                       base_onoff="~/TFM/5.on_off_body",
                       base_vanhees="~/TFM/6.Van_Hees"):
    """
    Load on/off-body masks and Van Hees inactivity masks,
    and combine them into a single binary mask.
    """
    base_onoff = os.path.expanduser(base_onoff)
    base_vanhees = os.path.expanduser(base_vanhees)

    def load_mask(base_dir, device_folder, subject_id):
        if "Van_Hees" in base_dir:
            filename = "ACC_sleep_masked.csv"
            source_col = "sleep_mask"
            target_col = "sleep"
        else:
            filename = "onbody_mask.csv"
            source_col = "is_onbody"
            target_col = "onbody"

        path = os.path.join(base_dir, device_folder, subject_id, filename)
        if not os.path.exists(path):
            raise FileNotFoundError(f"Mask not found: {path}")

        df = pd.read_csv(path)
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s", utc=True)
        return df.rename(columns={source_col: target_col})

    # Embrace masks
    on_e = load_mask(base_onoff, "Embrace_mask", embrace_id)
    vh_e = load_mask(base_vanhees, "Embrace_mask", embrace_id)

    mask_embrace = pd.merge_asof(
        on_e.sort_values("timestamp"),
        vh_e.sort_values("timestamp"),
        on="timestamp",
        direction="nearest",
        tolerance=pd.Timedelta("1s")
    )
    mask_embrace["combined"] = (
        (mask_embrace["onbody"] == 1) &
        (mask_embrace["sleep"] == 0)
    ).astype(int)

    # Empatica masks
    on_m = load_mask(base_onoff, "Empatica_mask", empatica_id)
    vh_m = load_mask(base_vanhees, "Empatica_mask", empatica_id)

    mask_empatica = pd.merge_asof(
        on_m.sort_values("timestamp"),
        vh_m.sort_values("timestamp"),
        on="timestamp",
        direction="nearest",
        tolerance=pd.Timedelta("1s")
    )
    mask_empatica["combined"] = (
        (mask_empatica["onbody"] == 1) &
        (mask_empatica["sleep"] == 0)
    ).astype(int)

    return mask_embrace, mask_empatica


# ============================================================
# 2. APPLY MASK TO A SIGNAL FILE (EDA, ACC OR IBI)
# ============================================================

def apply_mask_to_signal(csv_path, mask_df, out_path,
                         n_cols=1, is_acc=False, is_ibi=False):
    """
    Apply the combined mask to a signal file and save the masked output.
    """
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"Signal file not found: {csv_path}")

    df = pd.read_csv(csv_path, header=None)

    # --------------------------------------------------------
    # Case 1: EDA or ACC signals
    # --------------------------------------------------------
    if not is_ibi:
        start_time = float(df.iloc[0, 0])
        freq = float(df.iloc[1, 0])
        data = df.iloc[2:, :n_cols].astype(float).values

        timestamps = pd.to_datetime(
            start_time + np.arange(len(data)) / freq,
            unit="s", utc=True
        )

        sig_df = pd.DataFrame(
            data,
            columns=[f"col{i}" for i in range(1, n_cols + 1)]
        )
        sig_df["timestamp"] = timestamps

        merged = pd.merge_asof(
            sig_df.sort_values("timestamp"),
            mask_df.sort_values("timestamp"),
            on="timestamp",
            direction="nearest",
            tolerance=pd.Timedelta("1s")
        )
        merged["combined"] = merged["combined"].fillna(0)
        sig_df.loc[merged["combined"] == 0, sig_df.columns[:-1]] = np.nan

        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with open(out_path, "w") as f:
            if is_acc:
                f.write(",".join([f"{start_time:.6f}"] * n_cols) + "\n")
                f.write(",".join([f"{freq:.6f}"] * n_cols) + "\n")
            else:
                f.write(f"{start_time:.6f}\n")
                f.write(f"{freq:.6f}\n")

            np.savetxt(
                f,
                sig_df.iloc[:, :-1].to_numpy(),
                delimiter=",",
                fmt="%.9f"
            )

    # --------------------------------------------------------
    # Case 2: IBI signal
    # --------------------------------------------------------
    else:
        start_time = float(df.iloc[0, 0])
        arr = df.iloc[1:].astype(float).to_numpy()
        offsets = arr[:, 0]
        ibis = arr[:, 1]

        timestamps = pd.to_datetime(start_time + offsets, unit="s", utc=True)
        sig_df = pd.DataFrame({"IBI": ibis, "timestamp": timestamps})

        merged = pd.merge_asof(
            sig_df.sort_values("timestamp"),
            mask_df.sort_values("timestamp"),
            on="timestamp",
            direction="nearest",
            tolerance=pd.Timedelta("1s")
        )
        merged["combined"] = merged["combined"].fillna(0)
        sig_df.loc[merged["combined"] == 0, "IBI"] = np.nan

        offsets_out = (
            sig_df["timestamp"].astype("int64") / 1e9 - start_time
        )

        out_data = np.column_stack([offsets_out, sig_df["IBI"].to_numpy()])

        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with open(out_path, "w") as f:
            f.write(f"{start_time:.6f},IBI\n")
            np.savetxt(f, out_data, delimiter=",", fmt="%.9f")


# ============================================================
# 3. PROCESS A FULL SUBJECT (ALL SIGNALS)
# ============================================================

def process_subject(embrace_id, empatica_id):
    """
    Apply combined masks to all signals of one subject.
    """
    mask_embrace, mask_empatica = load_combined_mask(embrace_id, empatica_id)

    base_in_embrace = os.path.expanduser(f"~/TFM/2.step2_e4csv/{embrace_id}")
    base_in_empatica = os.path.expanduser(f"~/TFM/EMPATICA_all/{empatica_id}")
    base_out = os.path.expanduser("~/TFM/7.FLIRT_ready")

    out_embrace = os.path.join(base_out, "Embrace", embrace_id)
    out_empatica = os.path.join(base_out, "Empatica", empatica_id)
    os.makedirs(out_embrace, exist_ok=True)
    os.makedirs(out_empatica, exist_ok=True)

    # Embrace signals (multiple parts)
    embrace_sensors = {
        "EDA": ("EDA_filtrado", 1, False, False),
        "ACC": ("ACC_filtrado", 3, True, False),
        "IBI": ("IBI_filtrado", 2, False, True),
    }

    for sensor, (folder, ncols, is_acc, is_ibi) in embrace_sensors.items():
        sensor_dir = os.path.join(base_in_embrace, folder)
        if not os.path.exists(sensor_dir):
            continue

        parts = sorted([f for f in os.listdir(sensor_dir) if f.lower().startswith("part")])
        if not parts:
            parts = sorted([f for f in os.listdir(sensor_dir) if f.lower().startswith("ibi_part")])
        if not parts:
            continue

        for part in parts:
            in_path = os.path.join(sensor_dir, part)
            out_path = os.path.join(
                out_embrace,
                f"{sensor}_{part.replace('.csv', '_masked.csv')}"
            )
            apply_mask_to_signal(
                in_path, mask_embrace, out_path,
                n_cols=ncols, is_acc=is_acc, is_ibi=is_ibi
            )

    # Empatica signals (single file per sensor)
    empatica_files = {
        "EDA": (os.path.join(base_in_empatica, "EDA.csv"), 1, False, False),
        "ACC": (os.path.join(base_in_empatica, "ACC.csv"), 3, True, False),
        "IBI": (os.path.join(base_in_empatica, "IBI.csv"), 2, False, True),
    }

    for sensor, (path, ncols, is_acc, is_ibi) in empatica_files.items():
        if not os.path.exists(path):
            continue

        out_path = os.path.join(out_empatica, f"{sensor}_masked.csv")
        apply_mask_to_signal(
            path, mask_empatica, out_path,
            n_cols=ncols, is_acc=is_acc, is_ibi=is_ibi
        )


# ============================================================
# 4. MAIN LOOP (PROCESS ALL SUBJECTS)
# ============================================================

if __name__ == "__main__":
    for embrace_id, empatica_id in PAIRING.items():
        try:
            process_subject(embrace_id, empatica_id)
        except Exception as e:
            print(f"Error processing {embrace_id} / {empatica_id}: {e}")


========== PROCESANDO TODOS LOS SUJETOS ==========

➡ Procesando CLIDEM01 ↔ 1823370
📂 Leyendo onbody_mask.csv de Embrace_mask/CLIDEM01
📂 Leyendo ACC_sleep_masked.csv de Embrace_mask/CLIDEM01
📂 Leyendo onbody_mask.csv de Empatica_mask/1823370
📂 Leyendo ACC_sleep_masked.csv de Empatica_mask/1823370
   🔄 EDA: 2 partes
   🧩 Aplicando máscara a part01.csv
   ✅ Guardado /home/yasmina/TFM/7.FLIRT_ready/Embrace/CLIDEM01/EDA_part01_masked.csv
   🧩 Aplicando máscara a part02.csv
   ✅ Guardado /home/yasmina/TFM/7.FLIRT_ready/Embrace/CLIDEM01/EDA_part02_masked.csv
   🔄 ACC: 1 partes
   🧩 Aplicando máscara a part01.csv
   ✅ Guardado /home/yasmina/TFM/7.FLIRT_ready/Embrace/CLIDEM01/ACC_part01_masked.csv
   🔄 IBI: 1 partes
   🧩 Aplicando máscara a IBI_part01.csv
   ✅ Guardado /home/yasmina/TFM/7.FLIRT_ready/Embrace/CLIDEM01/IBI_IBI_part01_masked.csv
   🧩 Aplicando máscara a EDA.csv
   ✅ Guardado /home/yasmina/TFM/7.FLIRT_ready/Empatica/1823370/EDA_masked.csv
   🧩 Aplicando máscara a ACC.csv
   ✅ Gua

**Automated selection of optimal FLIRT segments**

In [21]:
# ============================================================
# ADVANCED DETECTION OF OPTIMAL SEGMENTS FOR FLIRT
# Automated processing for all subjects (Logic: Option A)
# ============================================================

import os
import numpy as np
import pandas as pd

# Base directory containing FLIRT-ready signals
base_dir = os.path.expanduser("~/TFM/7.FLIRT_ready")

# ============================================================
# SUBJECT PAIRING: Embrace Plus → Empatica
# ============================================================

PAIRING = {
    "CLIDEM01": "1823370", "CLIDEM010": "1842113", "CLIDEM011": "1891004",
    "CLIDEM02": "1832743", "CLIDEM030": "1916330", "CLIDEM031": "1920720",
    "CLIDEM04": "1833368", "CLIDEM05": "1837341", "CLIDEM06": "1840660",
    "CLIDEM07": "1840605", "CLIDEM08": "1841430", "CLIDEM09": "1842162",
    "CLIDEM11": "1891004", "CLIDEM12": "1891236", "CLIDEM13": "1895868",
    "CLIDEM14": "1891785", "CLIDEM15": "1894131", "CLIDEM16": "1895849",
    "CLIDEM17": "1896649", "CLIDEM18": "1898066", "CLIDEM19": "1900189",
    "CLIDEM20": "1906938", "CLIDEM21": "1907236", "CLIDEM22": "1908525",
    "CLIDEM23": "1909384", "CLIDEM24": "1910353", "CLIDEM25": "1911867",
    "CLIDEM26": "1912600", "CLIDEM27": "1913365", "CLIDEM29": "1914654"
}

# ============================================================
# PARAMETERS
# ============================================================

MIN_SEGMENT_MIN = 10                 # Minimum segment duration (minutes)
MAX_GAP_SEC = 10                     # Max interpolation gap (seconds)
MIN_CONTINUITY_MINUTE = 0.90         # Minimum continuity per minute
MIN_LATIDOS_MINUTE = 10              # Minimum beats per minute
MIN_IBI_TOTAL_SEGMENT = 200          # Minimum IBI count per segment

# Scoring weights
W_ACC_VAR = 0.40
W_EDA_VAR = 0.25
W_HRV_VAR = 0.25
W_CONT = 0.10

# ============================================================
# ROBUST CSV LOADER FOR MASKED SIGNALS
# ============================================================

def load_masked_signal(path, sensor):
    """
    Load a masked signal CSV (EDA, ACC or IBI) and return a DataFrame
    indexed by UTC timestamp.
    """
    df = pd.read_csv(path, header=None, engine="python", on_bad_lines="skip")

    # IBI format
    if sensor.startswith("IBI"):
        start_time = float(df.iloc[0, 0])
        df = df.iloc[1:].apply(pd.to_numeric, errors="coerce").dropna()

        if df.shape[1] < 2:
            raise ValueError(f"Invalid IBI format: {path}")

        df = df.iloc[:, :2]
        df.columns = ["offset", "value"]
        df["timestamp"] = pd.to_datetime(start_time + df["offset"], unit="s", utc=True)
        return df.set_index("timestamp")[["value"]]

    # EDA / ACC format
    start_time = float(df.iloc[0, 0])
    freq = float(df.iloc[1, 0])

    data = df.iloc[2:].apply(pd.to_numeric, errors="coerce").dropna().values
    timestamps = start_time + np.arange(len(data)) / freq

    out = pd.DataFrame(
        data,
        columns=[f"col{i}" for i in range(1, data.shape[1] + 1)]
    )
    out["timestamp"] = pd.to_datetime(timestamps, unit="s", utc=True)

    return out.set_index("timestamp")


def resample_continuous(df):
    """
    Resample signal to 1 Hz and interpolate short gaps.
    """
    return df.resample("1s").mean().interpolate(
        limit=MAX_GAP_SEC,
        limit_direction="both"
    )


def load_all(embrace_id, empatica_id, sensor):
    """
    Load masked signals for both devices and a given sensor.
    """
    path_empatica = os.path.join(
        base_dir, "Empatica", empatica_id, f"{sensor}_masked.csv"
    )
    embrace_dir = os.path.join(base_dir, "Embrace", embrace_id)

    embrace_files = sorted([
        os.path.join(embrace_dir, f)
        for f in os.listdir(embrace_dir)
        if f.lower().startswith(sensor.lower()) and f.endswith("_masked.csv")
    ])

    df_embrace = pd.concat(
        [load_masked_signal(p, sensor) for p in embrace_files]
    ).sort_index()

    df_empatica = load_masked_signal(path_empatica, sensor)

    return df_embrace, df_empatica

# ============================================================
# BEST LONG SEGMENT DETECTION (OPTION A)
# ============================================================

def detect_best_long_segment(embrace_id, empatica_id):
    """
    Detect the best continuous segment according to continuity,
    signal variability and HRV metrics.
    """
    eda_e, eda_m = load_all(embrace_id, empatica_id, "EDA")
    acc_e, acc_m = load_all(embrace_id, empatica_id, "ACC")
    ibi_e, ibi_m = load_all(embrace_id, empatica_id, "IBI")

    eda_e_r, eda_m_r = resample_continuous(eda_e), resample_continuous(eda_m)
    acc_e_r, acc_m_r = resample_continuous(acc_e), resample_continuous(acc_m)

    t_start = max(
        eda_e_r.index.min(), acc_e_r.index.min(),
        eda_m_r.index.min(), acc_m_r.index.min()
    )
    t_end = min(
        eda_e_r.index.max(), acc_e_r.index.max(),
        eda_m_r.index.max(), acc_m_r.index.max()
    )

    one_min = pd.Timedelta(minutes=1)
    current = t_start
    minute_stats = []

    # Evaluate each minute
    while current + one_min <= t_end:
        s, e = current, current + one_min
        idx = pd.date_range(s, e, freq="1s", tz="UTC")

        continuity = np.mean([
            eda_e_r.reindex(idx).notna().mean().iloc[0],
            acc_e_r.reindex(idx).notna().mean().iloc[0],
            eda_m_r.reindex(idx).notna().mean().iloc[0],
            acc_m_r.reindex(idx).notna().mean().iloc[0]
        ])

        lat_e = ibi_e[(ibi_e.index >= s) & (ibi_e.index < e)]
        lat_m = ibi_m[(ibi_m.index >= s) & (ibi_m.index < e)]

        valid = (
            continuity >= MIN_CONTINUITY_MINUTE and
            len(lat_e) >= MIN_LATIDOS_MINUTE and
            len(lat_m) >= MIN_LATIDOS_MINUTE
        )

        minute_stats.append((s, e, valid, continuity, len(lat_e), len(lat_m)))
        current += one_min

    # Group consecutive valid minutes
    segments = []
    buffer = []
    seg_start = None

    for s, e, valid, cont, le, lm in minute_stats:
        if valid:
            if seg_start is None:
                seg_start = s
            buffer.append((s, e, cont, le, lm))
        else:
            if seg_start:
                segments.append((seg_start, buffer[-1][1], buffer))
            seg_start = None
            buffer = []

    if seg_start:
        segments.append((seg_start, buffer[-1][1], buffer))

    # Score segments
    scored_segments = []

    for seg_start, seg_end, mins in segments:
        duration = (seg_end - seg_start).total_seconds() / 60
        if duration < MIN_SEGMENT_MIN:
            continue

        cont_mean = np.mean([m[2] for m in mins])
        lat_e_total = sum(m[3] for m in mins)
        lat_m_total = sum(m[4] for m in mins)

        if lat_e_total < MIN_IBI_TOTAL_SEGMENT or lat_m_total < MIN_IBI_TOTAL_SEGMENT:
            continue

        # ACC variability (L2 norm)
        def l2(df):
            return np.linalg.norm(df[["col1", "col2", "col3"]].values, axis=1)

        acc_var = np.mean([
            np.var(l2(acc_e[(acc_e.index >= seg_start) & (acc_e.index <= seg_end)])),
            np.var(l2(acc_m[(acc_m.index >= seg_start) & (acc_m.index <= seg_end)]))
        ])

        # EDA variability
        eda_var = np.mean([
            eda_e.loc[seg_start:seg_end]["col1"].var(),
            eda_m.loc[seg_start:seg_end]["col1"].var()
        ])

        # HRV variability
        hrv_var = np.mean([
            ibi_e.loc[seg_start:seg_end]["value"].var(),
            ibi_m.loc[seg_start:seg_end]["value"].var()
        ])

        score = (
            W_ACC_VAR * acc_var +
            W_EDA_VAR * eda_var +
            W_HRV_VAR * hrv_var +
            W_CONT * cont_mean
        ) * np.log1p(duration)

        scored_segments.append((seg_start, seg_end, duration, score))

    if not scored_segments:
        return None

    return max(scored_segments, key=lambda x: x[3])

# ============================================================
# EXPORT SEGMENT IN FLIRT FORMAT
# ============================================================

def save_flirt(sensor, source, df, t_start, t_end, out_dir, subject_id):
    """
    Save a signal segment in FLIRT-compatible format.
    """
    df_cut = df.loc[t_start:t_end]
    if df_cut.empty:
        return

    out_path = os.path.join(out_dir, f"{source}_{sensor}_FLIRT_{subject_id}.csv")

    if sensor == "EDA":
        t0 = df_cut.index[0].timestamp()
        with open(out_path, "w") as f:
            f.write(f"{t0}\n4\n")
            for v in df_cut["col1"]:
                f.write(f"{v}\n")

    elif sensor == "ACC":
        t0 = df_cut.index[0].timestamp()
        with open(out_path, "w") as f:
            f.write(f"{t0},{t0},{t0}\n32,32,32\n")
            for _, r in df_cut.iterrows():
                f.write(f"{r['col1']},{r['col2']},{r['col3']}\n")

    elif sensor == "IBI":
        t0 = df_cut.index[0].timestamp()
        with open(out_path, "w") as f:
            f.write(f"{t0}\n")
            for ts, val in df_cut["value"].items():
                f.write(f"{ts.timestamp() - t0},{val}\n")


def export_segment(embrace_id, empatica_id, best_segment):
    """
    Export the selected best segment for both devices and all sensors.
    """
    t_start, t_end, _, _ = best_segment
    subject_id = f"{embrace_id}_{empatica_id}"

    out_embrace = os.path.join(base_dir, "EmbracePlus_out", subject_id)
    out_empatica = os.path.join(base_dir, "Empatica_out", subject_id)
    os.makedirs(out_embrace, exist_ok=True)
    os.makedirs(out_empatica, exist_ok=True)

    eda_e, eda_m = load_all(embrace_id, empatica_id, "EDA")
    acc_e, acc_m = load_all(embrace_id, empatica_id, "ACC")
    ibi_e, ibi_m = load_all(embrace_id, empatica_id, "IBI")

    save_flirt("EDA", "Embrace", eda_e, t_start, t_end, out_embrace, subject_id)
    save_flirt("EDA", "Empatica", eda_m, t_start, t_end, out_empatica, subject_id)

    save_flirt("ACC", "Embrace", acc_e, t_start, t_end, out_embrace, subject_id)
    save_flirt("ACC", "Empatica", acc_m, t_start, t_end, out_empatica, subject_id)

    save_flirt("IBI", "Embrace", ibi_e, t_start, t_end, out_embrace, subject_id)
    save_flirt("IBI", "Empatica", ibi_m, t_start, t_end, out_empatica, subject_id)

# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":

    print("Processing all subjects")

    for embrace_id, empatica_id in PAIRING.items():
        subject_id = f"{embrace_id}_{empatica_id}"

        try:
            best_segment = detect_best_long_segment(embrace_id, empatica_id)

            if best_segment is None:
                print(f"{subject_id}: no valid segment found")
                continue

            duration = best_segment[2]
            print(f"{subject_id}: best segment duration = {duration:.2f} minutes")

            export_segment(embrace_id, empatica_id, best_segment)

        except Exception as e:
            print(f"{subject_id}: error ({e})")


=========== PROCESANDO TODOS LOS SUJETOS ===========

CLIDEM01_1823370: SIN TRAMOS VÁLIDOS
CLIDEM010_1842113: SIN TRAMOS VÁLIDOS
CLIDEM011_1891004: Mejor tramo = 18.00 minutos
CLIDEM02_1832743: Mejor tramo = 12.00 minutos
CLIDEM030_1916330: Mejor tramo = 15.00 minutos
CLIDEM031_1920720: SIN TRAMOS VÁLIDOS
CLIDEM04_1833368: SIN TRAMOS VÁLIDOS
CLIDEM05_1837341: Mejor tramo = 12.00 minutos
CLIDEM06_1840660: Mejor tramo = 13.00 minutos
CLIDEM07_1840605: Mejor tramo = 18.00 minutos
CLIDEM08_1841430: Mejor tramo = 13.00 minutos
CLIDEM09_1842162: SIN TRAMOS VÁLIDOS
CLIDEM11_1891004: Mejor tramo = 18.00 minutos
CLIDEM12_1891236: Mejor tramo = 14.00 minutos
CLIDEM13_1895868: Mejor tramo = 18.00 minutos
CLIDEM14_1891785: Mejor tramo = 10.00 minutos
CLIDEM15_1894131: Mejor tramo = 12.00 minutos
CLIDEM16_1895849: Mejor tramo = 20.00 minutos
CLIDEM17_1896649: Mejor tramo = 12.00 minutos
CLIDEM18_1898066: Mejor tramo = 11.00 minutos
CLIDEM19_1900189: Mejor tramo = 10.00 minutos
CLIDEM20_1906938: Me

**Masked signal comparison (EDA, ACC, IBI)**

In [22]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ============================================================
# === SUBJECT → EMPATICA PAIRING =============================
# ============================================================

PAIRING = {
    "CLIDEM01": "1823370", "CLIDEM010": "1842113", "CLIDEM011": "1891004",
    "CLIDEM02": "1832743", "CLIDEM030": "1916330", "CLIDEM031": "1920720",
    "CLIDEM04": "1833368", "CLIDEM05": "1837341", "CLIDEM06": "1840660",
    "CLIDEM07": "1840605", "CLIDEM08": "1841430", "CLIDEM09": "1842162",
    "CLIDEM11": "1891004", "CLIDEM12": "1891236", "CLIDEM13": "1895868",
    "CLIDEM14": "1891785", "CLIDEM15": "1894131", "CLIDEM16": "1895849",
    "CLIDEM17": "1896649", "CLIDEM18": "1898066", "CLIDEM19": "1900189",
    "CLIDEM20": "1906938", "CLIDEM21": "1907236", "CLIDEM22": "1908525",
    "CLIDEM23": "1909384", "CLIDEM24": "1910353", "CLIDEM25": "1911867",
    "CLIDEM26": "1912600", "CLIDEM27": "1913365", "CLIDEM29": "1914654"
}

# ============================================================
# === SINGLE FILE READERS ====================================
# ============================================================

def read_signal_file(path, n_cols=1, is_acc=False, is_ibi=False):
    """
    Generic reader for masked signal files (EDA, ACC, IBI).
    Handles Empatica and Embrace formats transparently.
    """
    df = pd.read_csv(path, header=None, na_values=["NA", "NaN", ""])

    # --- IBI ---
    if is_ibi:
        try:
            start_time = float(df.iloc[0, 0])

            # Empatica format: base timestamp + relative offsets
            if df.shape[1] == 2 and np.all(df.iloc[1:, 0].astype(float) < 1e7):
                ibi_data = df.iloc[1:].astype(float)
                ibi_data.columns = ["timestamp_rel", "IBI"]
                ibi_data["timestamp_abs"] = start_time + ibi_data["timestamp_rel"]
                ibi_data["time"] = pd.to_datetime(
                    ibi_data["timestamp_abs"], unit="s", utc=True
                )
                return ibi_data[["time", "IBI"]]
        except Exception:
            pass

        # Embrace format: absolute timestamp + value
        if df.shape[1] >= 2:
            df = df.iloc[:, :2].copy()
            df.columns = ["timestamp", "IBI"]
            df["time"] = pd.to_datetime(df["timestamp"], unit="s", utc=True)
            return df[["time", "IBI"]]

        raise ValueError(f"Unknown IBI format: {path}")

    # --- ACC ---
    if is_acc:
        start_time = float(df.iloc[0, 0])
        freq = float(df.iloc[1, 0])
        vals = df.iloc[2:, :3].astype(float).values
        ts = start_time + np.arange(len(vals)) / freq
        mag = np.linalg.norm(vals, axis=1)
        return pd.DataFrame({
            "time": pd.to_datetime(ts, unit="s", utc=True),
            "ACC": mag
        })

    # --- EDA or other 1D signals ---
    start_time = float(df.iloc[0, 0])
    freq = float(df.iloc[1, 0])
    vals = df.iloc[2:, 0].astype(float).values
    ts = start_time + np.arange(len(vals)) / freq
    return pd.DataFrame({
        "time": pd.to_datetime(ts, unit="s", utc=True),
        "value": vals
    })


# ============================================================
# === LOAD ALL EMBRACE PARTS =================================
# ============================================================

def load_embrace_parts(base_dir, signal, n_cols=1, is_acc=False, is_ibi=False):
    """
    Automatically locates and concatenates all masked Embrace files:
    EDA_partXX_masked.csv, ACC_partXX_masked.csv, IBI_partXX_masked.csv
    """
    if not os.path.exists(base_dir):
        print(f"Embrace directory does not exist: {base_dir}")
        return None

    parts = sorted([
        os.path.join(base_dir, f)
        for f in os.listdir(base_dir)
        if f.lower().startswith(signal.lower()) and f.lower().endswith("_masked.csv")
    ])

    if not parts:
        print(f"No masked parts found for {signal} in {base_dir}")
        return None

    dfs = []
    for p in parts:
        try:
            dfs.append(read_signal_file(
                p, n_cols=n_cols, is_acc=is_acc, is_ibi=is_ibi
            ))
        except Exception as e:
            print(f"Error reading {p}: {e}")

    if not dfs:
        return None

    return pd.concat(dfs).sort_values("time").reset_index(drop=True)


# ============================================================
# === DOWNSAMPLE TO 1 HZ =====================================
# ============================================================

def downsample(df, colname):
    df = df.copy()
    df["sec"] = df["time"].dt.floor("1s")
    return (
        df.groupby("sec", as_index=False)[colname]
        .mean()
        .rename(columns={"sec": "time"})
    )


# ============================================================
# === MAIN PLOTTING FUNCTION =================================
# ============================================================

def plot_masked_signals_fixed_ibi(empatica_id,
                                  embrace_id,
                                  base_masked="~/TFM/7.FLIRT_ready",
                                  save_fig=True):
    """
    Plot masked EDA, ACC and IBI signals for Empatica and Embrace.
    IBI is not downsampled; continuous signals are downsampled to 1 Hz.
    """
    base_masked = os.path.expanduser(base_masked)
    base_embrace = os.path.join(base_masked, "Embrace", embrace_id)
    base_empatica = os.path.join(base_masked, "Empatica", empatica_id)

    files = {
        "EDA": ("EDA_masked.csv", 1, False, False),
        "ACC": ("ACC_masked.csv", 3, True, False),
        "IBI": ("IBI_masked.csv", 1, False, True)
    }

    signals_e, signals_m = {}, {}

    for name, (fname, ncols, is_acc, is_ibi) in files.items():
        # Empatica
        path_e = os.path.join(base_empatica, fname)
        if not os.path.exists(path_e):
            print(f"Missing Empatica file {fname} for {empatica_id}")
            continue
        df_e = read_signal_file(path_e, n_cols=ncols, is_acc=is_acc, is_ibi=is_ibi)

        # Embrace
        df_m = load_embrace_parts(
            base_embrace, name, n_cols=ncols, is_acc=is_acc, is_ibi=is_ibi
        )
        if df_m is None:
            continue

        colname = "ACC" if is_acc else ("IBI" if is_ibi else "value")

        # Downsample only continuous signals
        if not is_ibi:
            df_e = downsample(df_e, colname)
            df_m = downsample(df_m, colname)

        signals_e[name], signals_m[name] = df_e, df_m

    if not signals_e:
        print(f"No valid signals for {embrace_id}_{empatica_id}")
        return None

    # ========================================================
    # === COMPARATIVE PLOT ===================================
    # ========================================================

    fig, axs = plt.subplots(3, 1, figsize=(16, 9), sharex=False)
    fig.suptitle(
        f"Masked signals – {embrace_id} / {empatica_id}",
        fontsize=14,
        fontweight="bold"
    )

    # EDA
    if "EDA" in signals_e:
        axs[0].plot(
            signals_e["EDA"]["time"],
            signals_e["EDA"]["value"],
            linewidth=0.8,
            label=f"Empatica {empatica_id}"
        )
        axs[0].plot(
            signals_m["EDA"]["time"],
            signals_m["EDA"]["value"],
            linewidth=0.8,
            label=f"Embrace {embrace_id}"
        )
        axs[0].set_ylabel("EDA (µS)")
        axs[0].legend()

    # ACC
    if "ACC" in signals_e:
        axs[1].plot(
            signals_e["ACC"]["time"],
            signals_e["ACC"]["ACC"],
            linewidth=0.8
        )
        axs[1].plot(
            signals_m["ACC"]["time"],
            signals_m["ACC"]["ACC"],
            linewidth=0.8
        )
        axs[1].set_ylabel("ACC |L2|")

    # IBI
    if "IBI" in signals_e:
        axs[2].plot(
            signals_e["IBI"]["time"],
            signals_e["IBI"]["IBI"],
            linewidth=0.8
        )
        axs[2].plot(
            signals_m["IBI"]["time"],
            signals_m["IBI"]["IBI"],
            linewidth=0.8
        )
        axs[2].set_ylabel("IBI (s)")
        axs[2].set_xlabel("Time (UTC)")
        axs[2].set_facecolor("whitesmoke")

    for ax in axs:
        ax.grid(alpha=0.3)

    plt.tight_layout()

    if save_fig:
        out_dir = os.path.join(base_masked, "plots_masked_signals")
        os.makedirs(out_dir, exist_ok=True)
        out_path = os.path.join(
            out_dir,
            f"compare_masked_FIXED_IBI_{embrace_id}_{empatica_id}.png"
        )
        plt.savefig(out_path, dpi=150)
        plt.close(fig)
        return out_path
    else:
        plt.show()
        return None


# ============================================================
# === MAIN: PROCESS ALL SUBJECTS =============================
# ============================================================

if __name__ == "__main__":
    base_masked = "~/TFM/7.FLIRT_ready"

    print("\nGenerating plots for all subjects\n")

    for embrace_id, empatica_id in PAIRING.items():
        try:
            out_path = plot_masked_signals_fixed_ibi(
                empatica_id=empatica_id,
                embrace_id=embrace_id,
                base_masked=base_masked,
                save_fig=True
            )
            if out_path:
                print(f"{embrace_id}_{empatica_id}: figure saved at {out_path}")
            else:
                print(f"{embrace_id}_{empatica_id}: no figure generated")
        except Exception as e:
            print(f"{embrace_id}_{empatica_id}: ERROR ({e})")

    print("\nAll plots completed\n")


=========== GENERANDO GRÁFICAS PARA TODOS LOS SUJETOS ===========

CLIDEM01_1823370: figura guardada en /home/yasmina/TFM/7.FLIRT_ready/plots_masked_signals/compare_masked_FIXED_IBI_CLIDEM01_1823370.png
CLIDEM010_1842113: figura guardada en /home/yasmina/TFM/7.FLIRT_ready/plots_masked_signals/compare_masked_FIXED_IBI_CLIDEM010_1842113.png
CLIDEM011_1891004: figura guardada en /home/yasmina/TFM/7.FLIRT_ready/plots_masked_signals/compare_masked_FIXED_IBI_CLIDEM011_1891004.png
CLIDEM02_1832743: figura guardada en /home/yasmina/TFM/7.FLIRT_ready/plots_masked_signals/compare_masked_FIXED_IBI_CLIDEM02_1832743.png
CLIDEM030_1916330: figura guardada en /home/yasmina/TFM/7.FLIRT_ready/plots_masked_signals/compare_masked_FIXED_IBI_CLIDEM030_1916330.png
CLIDEM031_1920720: figura guardada en /home/yasmina/TFM/7.FLIRT_ready/plots_masked_signals/compare_masked_FIXED_IBI_CLIDEM031_1920720.png
CLIDEM04_1833368: figura guardada en /home/yasmina/TFM/7.FLIRT_ready/plots_masked_signals/compare_masked_FIX

**FLIRT Feature Extraction per Sensor**

In [24]:
# ============================================================
# ===     FLIRT PER SENSOR — AUTOMATED FOR ALL SUBJECTS
# ============================================================

import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np
import flirt
import matplotlib.pyplot as plt

BASE = os.path.expanduser("~/TFM/7.FLIRT_ready")


# ============================================================
# === DYNAMIC CONFIGURATION BY DEVICE ========================
# ============================================================

def elegir_configuracion(dispositivo, n_ibi):
    """
    Selects FLIRT window configuration depending on the device.
    Empatica uses shorter HRV windows due to higher temporal resolution.
    """
    if dispositivo.lower().startswith("empatica"):
        ventana_hrv = 20
        paso_hrv = 1
    else:
        ventana_hrv = 60
        paso_hrv = 1

    return {
        "ventana_hrv": ventana_hrv,
        "paso_hrv": paso_hrv,
        "ventana_acc": 60,
        "paso_acc": 1,
        "ventana_eda": 60,
        "paso_eda": 1,
    }


# ============================================================
# === FLIRT FILE READERS (IBI / EDA / ACC) ===================
# ============================================================

def load_ibi_flirt(path):
    """
    Loads an IBI file in FLIRT-compatible format.
    First line contains the absolute start timestamp.
    Subsequent lines contain relative offsets and IBI values.
    """
    rows = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        lines = [l.strip().replace("\ufeff", "") for l in f if l.strip()]

    t0 = float(lines[0])

    for line in lines[1:]:
        parts = [p.strip() for p in line.replace(";", ",").split(",")]
        if len(parts) != 2:
            continue

        try:
            offset = float(parts[0])
            ibi = float(parts[1])
            rows.append([t0 + offset, ibi])
        except Exception:
            pass

    df = pd.DataFrame(rows, columns=["timestamp", "ibi"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s", utc=True)
    df = df.drop_duplicates("timestamp")

    # Physiological plausibility filter
    df = df[(df["ibi"] > 0.2) & (df["ibi"] < 3.0)]

    return df.set_index("timestamp")


def load_eda_flirt(path):
    """
    Loads an EDA signal in FLIRT format.
    First line: start timestamp
    Second line: sampling frequency
    Remaining lines: EDA values
    """
    df_raw = pd.read_csv(path, header=None)
    start_ts = float(df_raw.iloc[0, 0])
    freq = float(df_raw.iloc[1, 0])
    values = df_raw.iloc[2:, 0].astype(float).values
    ts = start_ts + np.arange(len(values)) / freq

    return pd.DataFrame(
        {"eda": values},
        index=pd.to_datetime(ts, unit="s", utc=True)
    )


def load_acc_flirt(path):
    """
    Loads an ACC signal in FLIRT format.
    First line: start timestamp (repeated for x,y,z)
    Second line: sampling frequency
    Remaining lines: x, y, z acceleration values
    """
    df_raw = pd.read_csv(path, header=None)
    start_ts = float(df_raw.iloc[0, 0])
    freq = float(df_raw.iloc[1, 0])
    values = df_raw.iloc[2:, :3].astype(float).values
    ts = start_ts + np.arange(len(values)) / freq

    df = pd.DataFrame(values, columns=["acc_x", "acc_y", "acc_z"])
    df["timestamp"] = pd.to_datetime(ts, unit="s", utc=True)

    return df.set_index("timestamp")


# ============================================================
# === RUN FLIRT FOR A SINGLE DEVICE ==========================
# ============================================================

def ejecutar_flirt_sensor(ruta_base, dispositivo, subject_folder):
    """
    Executes FLIRT feature extraction for EDA, ACC and IBI
    for a given subject and device.
    """

    ruta = os.path.join(ruta_base, subject_folder)

    ruta_eda = os.path.join(ruta, f"{dispositivo}_EDA_FLIRT_{subject_folder}.csv")
    ruta_acc = os.path.join(ruta, f"{dispositivo}_ACC_FLIRT_{subject_folder}.csv")
    ruta_ibi = os.path.join(ruta, f"{dispositivo}_IBI_FLIRT_{subject_folder}.csv")

    if not (os.path.exists(ruta_eda) and os.path.exists(ruta_acc) and os.path.exists(ruta_ibi)):
        print(f"Missing FLIRT input signals for {dispositivo} in {subject_folder}")
        return None, None, None

    print("\n====================================")
    print(f"FLIRT per sensor - {dispositivo} - {subject_folder}")
    print("====================================")

    df_eda = load_eda_flirt(ruta_eda)
    df_acc = load_acc_flirt(ruta_acc)
    df_ibi = load_ibi_flirt(ruta_ibi)

    cfg = elegir_configuracion(dispositivo, len(df_ibi))

    # ---------------- HRV ----------------
    if len(df_ibi) < cfg["ventana_hrv"]:
        print("HRV skipped: insufficient number of IBI samples")
        hrv = None
    else:
        hrv = flirt.get_hrv_features(
            df_ibi["ibi"],
            window_length=cfg["ventana_hrv"],
            window_step_size=cfg["paso_hrv"],
            domains=["td", "fd", "stat"],
            clean_data=False
        )

    # ---------------- ACC ----------------
    acc = flirt.get_acc_features(
        df_acc,
        window_length=cfg["ventana_acc"],
        window_step_size=cfg["paso_acc"],
        data_frequency=32
    )

    # ---------------- EDA ----------------
    eda = flirt.get_eda_features(
        df_eda["eda"],
        window_length=cfg["ventana_eda"],
        window_step_size=cfg["paso_eda"],
        data_frequency=4
    )

    # ---------------- SAVE OUTPUT ----------------
    out_dir = os.path.join(ruta, "FLIRT_features")
    os.makedirs(out_dir, exist_ok=True)

    if hrv is not None:
        hrv.to_csv(os.path.join(out_dir, f"FLIRT_HRV_{dispositivo}_{subject_folder}.csv"))

    acc.to_csv(os.path.join(out_dir, f"FLIRT_ACC_{dispositivo}_{subject_folder}.csv"))
    eda.to_csv(os.path.join(out_dir, f"FLIRT_EDA_{dispositivo}_{subject_folder}.csv"))

    print("Saved FLIRT features in:", out_dir)

    return hrv, acc, eda


# ============================================================
# ============================== MAIN ========================
# ============================================================

if __name__ == "__main__":

    print("\n=========== PROCESSING ALL SUBJECTS — FLIRT ===========\n")

    # Automatically detect subject folders generated by the previous FLIRT preparation step
    embrace_out = os.path.join(BASE, "EmbracePlus_out")
    empatica_out = os.path.join(BASE, "Empatica_out")

    subjects = sorted(os.listdir(embrace_out))

    for subject_folder in subjects:
        print(f"\n===== SUBJECT: {subject_folder} =====")

        ejecutar_flirt_sensor(embrace_out, "Embrace", subject_folder)
        ejecutar_flirt_sensor(empatica_out, "Empatica", subject_folder)

    print("\n================== END OF FLIRT EXTRACTION ==================\n")


=========== PROCESANDO TODOS LOS SUJETOS — FLIRT ===========


===== SUJETO: CLIDEM010_1842113 =====
⚠️  Falta señal FLIRT para Embrace en CLIDEM010_1842113
⚠️  Falta señal FLIRT para Empatica en CLIDEM010_1842113

===== SUJETO: CLIDEM011_1891004 =====

     FLIRT por sensor - Embrace - CLIDEM011_1891004


EDA features: 100%|██████████| 1081/1081 [00:00<00:00, 3760.96it/s]


✔ Guardado en: /home/yasmina/TFM/7.FLIRT_ready/EmbracePlus_out/CLIDEM011_1891004/FLIRT_features

     FLIRT por sensor - Empatica - CLIDEM011_1891004


EDA features: 100%|██████████| 1080/1080 [00:00<00:00, 3636.55it/s]


✔ Guardado en: /home/yasmina/TFM/7.FLIRT_ready/Empatica_out/CLIDEM011_1891004/FLIRT_features

===== SUJETO: CLIDEM01_1823370 =====
⚠️  Falta señal FLIRT para Embrace en CLIDEM01_1823370
⚠️  Falta señal FLIRT para Empatica en CLIDEM01_1823370

===== SUJETO: CLIDEM02_1832743 =====

     FLIRT por sensor - Embrace - CLIDEM02_1832743


...

✔ Guardado en: /home/yasmina/TFM/7.FLIRT_ready/Empatica_out/CLIDEM27_1913365/FLIRT_features

===== SUJETO: CLIDEM29_1914654 =====

     FLIRT por sensor - Embrace - CLIDEM29_1914654


EDA features: 100%|██████████| 660/660 [00:00<00:00, 4880.76it/s]


✔ Guardado en: /home/yasmina/TFM/7.FLIRT_ready/EmbracePlus_out/CLIDEM29_1914654/FLIRT_features

     FLIRT por sensor - Empatica - CLIDEM29_1914654


EDA features: 100%|██████████| 660/660 [00:00<00:00, 4678.57it/s]


✔ Guardado en: /home/yasmina/TFM/7.FLIRT_ready/Empatica_out/CLIDEM29_1914654/FLIRT_features

================== FIN DE EXTRACCIÓN FLIRT ==================

